# Make a Coadd from Gen3 Run3 DDF



See Bob Armstrong slides

https://docs.google.com/presentation/d/1bs3z6bqpAZ2WuSjpZOGuhgRnpz3Zd7ighkM-IDgk1dA/view

Run 2.2i DC2 data in:

```
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo
```

Create a user directory in the gen3 repo. This should be a symlink to write-accessible area [that you’ve created separately]:

```
/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo/u/<username>
```

Ask Jim for credentials to access to the postgres db

Run 3.1i DC2 DDF data in:

```
/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/
```

1. Create your own user directory under the above repo
   
```
/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/
```

E.g., `/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/wmwv`
    
as a symlink to a directory you've created on $SCRATCH to hold data.

```
mkdir -p $SCRATCH/DC2/Gen3/Run_3.1i/repo
ln -s $SCRATCH/DC2/Gen3/Run_3.1i/repo /global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo/u/wmwv
```



Bob's slide on Learning the new Butler

Science Pipeline Tutorial from DM: https://pipelines.lsst.io/v/weekly/

MWV: I found reading the following pretty key for starting to make my way around the Butler, but it also left me with questions:

Middleware tutorials:
  * https://pipelines.lsst.io/v/weekly/middleware/faq.html
  * https://pipelines.lsst.io/v/weekly/modules/lsst.daf.butler/organizing.html

Testing datasets: https://github.com/lsst/ci_imsim - A small dataset that allows you to run through the whole pipeline in a finite set of time.

Slack channels: #dm-middleware-support, #dm-middleware-dev

Tutorial by Jim Chiang on how to use parsl to scale processing on NERSC: https://docs.google.com/presentation/d/1EO_UBVhISBrBussCsIvJhNVxSnyfg5z6yIPaz99gA0A/view

Tutorial on creating a PipelineTask: https://github.com/lsst/pipe_base/blob/main/doc/lsst.pipe.base/creating-a-pipelinetask.rst

Also see  
https://pipelines.lsst.io/getting-started/dc2-guide.html


In [ ]:
from lsst.daf.butler import Butler

In [ ]:
# Run2.2i
repo_location = "/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo"
collection = "u/descdm/sfp_ddf_visits_part_01"
tract = 4848

# Run3.1i
# repo_location = "/global/cfs/cdirs/lsst/production/gen3/DC2/Run2.2i/repo"
# collection = "u/descdm/sfp_Y1_4431_4432_visits_part_00"
# tract = 4432

In [ ]:
butler = Butler(repo_location)

Here are all the types the Butler currently knows about.  These don't all necessarily exist.  They are the Dataset types that have been registered with the Butler.

In [ ]:
list(butler.registry.queryDatasetTypes())

In [ ]:
list(butler.registry.queryDatasetTypes(["calexp", "visitSummary"]))

This Butler doesn't know about coadds yet because no coadds have been run:

In [ ]:
list(butler.registry.queryDatasetTypes("goodSeeingCoadd"))

Here are the datasets, "collections" that the Butler knows about:

In [ ]:
list(butler.registry.queryCollections())

The entries we're interested in right now are are the collections of the form `u/descdm/sfp_ddf_visits_part_0[012]`.  In total there are/will be 10 parts.  Right now just the first three have been processed.

The "2.2i" entries are for the calibration images, because it's the same calibration data for 2.2i and 3.1i.

### Dimension Records

The various attributes for the data ID keys are called "dimensions".

In [ ]:
butler.registry.dimensions.getGovernorDimensions()

In [ ]:
print(butler.registry.dimensions["instrument"].RecordClass.fields)

But `skymap` doesn't work with a direct reference to print out the `fields`:

In [ ]:
print(butler.registry.dimensions["skymap"].RecordClass.fields)

One can get the names of these fields:

In [ ]:
print(butler.registry.dimensions["skymap"].RecordClass.fields.names)

One key dataset we're going to need from the single-frame processing is the `visitSummary`

In [ ]:
list(butler.registry.queryDatasetTypes("visitSummary"))

In [ ]:
i = 0
for data_ref in butler.registry.queryDatasets(
    datasetType="calexp",
    collections=collection,
    instrument="LSSTCam-imSim",
    where=f"tract={tract} and skymap='DC2'",
):
    i = i+1
    if i > 10:
         break
        
    print(data_ref.dataId.full)

In [ ]:
print("Required: ", data_ref.dataId.graph.required)
print("Implied: ", data_ref.dataId.graph.implied)

In [ ]:
butler.registry.expandDataId(data_ref.dataId).full

In [ ]:
data_ref.dimensions

In [ ]:
data_ref.dataId.full

In [ ]:
for data_id in butler.registry.queryDataIds(
    ["tract", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="calexp",
    collections=collection,
    where=f"skymap='DC2'",
).limit(10):
    print(data_id.full)

The `limit(10)` appears to work at some level before the final return because I only get 2.  If I do 100, I get more, although I didn't count them.

In [ ]:
collection

Queries are not de-duplicated.  Need to use `set` to de-duplicate.

In [ ]:
from collections import defaultdict
grouped_by_tract = defaultdict(set)

for data_id in butler.registry.queryDataIds(
    ["tract", "visit", "detector"],
    instrument="LSSTCam-imSim",
    datasets="calexp",
    collections=collection,
    where=f"tract={tract} and patch=37 and skymap='DC2'",
).limit(1000):
    grouped_by_tract[data_id["tract"]].add(data_id)

In [ ]:
print([f"{k}: {len(v)} matching dataIds" for k, v in grouped_by_tract.items()])                              

The LSST Science Pipelines Data Release Processing:

https://github.com/lsst/drp_pipe
https://github.com/lsst/drp_pipe/blob/main/ingredients/DRP.yaml


From Lee Kelvin's writeup:  

"""  
Coadd-processing takes place within step3 of the DRP.yaml pipeline. A large number of tasks are performed during this step, including, but not limited to:

jointcal - astrometric and photometric calibration  
makeWarp - warping of visit-level imaging onto the coadd-plane  
assembleCoadd - coaddition of warped visit-level imaging into coadded images  
detection - coadd-level object detection  
deblend - object deblending  
writeObjectTable - construction of coadd-level object catalogue  
"""

You can use the command-line version of the `butler` query tools by launcing a matching environment:

On the command line:
```
shifter --clearenv --image=lsstdesc/td-env:dev /bin/bash
bash-4.2$ source /opt/lsst/software/stack/loadLSST.bash
ebash-4.2$ setup lsst_distrib
```

```
REPO=/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo
```

Make a YAML file describing what we want to happen
subsets:
    dia
    - selectListVisits
    - templateGen


Running the following command-line task would do the warping for 125 images to prepare them to make a coadd for (tract, patch) (4848, 37).  This took about 60 minutes to run in serial using 8 cores on a Cori login node.

```
! pipetask run -b $REPO/butler.yaml \
-d "tract = 4848 AND skymap = 'DC2' AND patch = 37 AND band = 'r'" \
-p $DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml#makeWarp \
-i u/descdm/sfp_ddf_visits_part_01 \
-o u/wmwv/warps \
--register-dataset-types \
2>&1 | tee makeWarp.log
```

```
! pipetask run -b $REPO/butler.yaml \
-d "tract = 4848 AND skymap = 'DC2' AND patch = 37 AND band = 'r'" \
-p $DRP_PIPE_DIR/ingredients/LSSTCam-imSim/DRP.yaml#assembleCoadd \
-i u/wmwv/warps \
-o u/wmwv/coadd \
--register-dataset-types \
2>&1 | tee assembleCoadd.log
```

Here we can run the first set of steps to get a basic set of coadds and then make a "best-seeing" template.

```
cat > coadd.yaml
description: DRP specialized for ImSim-DC2 data
instrument: lsst.obs.lsst.LsstCamImSim
imports:
  - $DRP_PIPE_DIR/ingredients/DRP.yaml
  - $FARO_DIR/pipelines/metrics_pipeline.yaml
subsets:
  coadd_steps:
    subset:
      - makeWarp
      - assembleCoadd
      - detection
      - selectGoodSeeingVisits
      - templateGen
    description: >
      Basic steps to get to template generation.
```

```
! REPO=/global/cfs/cdirs/lsst/production/gen3/DC2/Run3.1i/repo; pipetask run -b $REPO/butler.yaml \
-d "tract = 4848 AND skymap = 'DC2' AND patch = 37 AND band = 'r'" \
-p coadd.yaml#coadd_steps \
-i u/descdm/sfp_ddf_visits_part_01 \
-o u/wmwv/coadds \
--register-dataset-types \
2>&1 | tee coadd_steps.log
```

I only know how to do the above reasonably through the command-line interface.

But we can do the `selectGoodSeeingVisits` through the API